# DECIDE – Pipeline LLM para classificação de queries (Groq API)

Este notebook replica o ficheiro `pipeline_decide.py` mas em formato interativo,
para podermos:
- testar cada passo,
- inspecionar DataFrames,
- alterar parâmetros (modelo, batch size, prompt, etc.).

### Ambiente recomendado

```bash
mamba create -n decide_env python=3.10
mamba activate decide_env
pip install groq pandas python-dotenv openpyxl


In [ ]:
# IMPORTS

import os
import time
import json
import re
import unicodedata
import pandas as pd

from dotenv import load_dotenv
from groq import Groq
from datetime import datetime

import logging
from logging.handlers import RotatingFileHandler

# ============================================================
# LOGGING CONFIGURATION
# ============================================================

log_filename = datetime.now().strftime("pipeline_run_%Y-%m-%d_%H-%M-%S.log")


log_handler = RotatingFileHandler(
    log_filename,
    maxBytes=5_000_000,
    backupCount=3,
    encoding="utf-8"
)



formatter = logging.Formatter(
    fmt="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

log_handler.setFormatter(formatter)

logger = logging.getLogger("DECIDE_PIPELINE")
logger.setLevel(logging.INFO)
logger.addHandler(log_handler)

console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# ============================================================
# CONFIGURAR GROQ API
# ============================================================

load_dotenv()

api_key = os.environ.get("GROQ_API_KEY")
if not api_key:
    raise ValueError("Variável de ambiente GROQ_API_KEY não definida.")

client = Groq(api_key=api_key)

MODEL_NAME = "llama-3.3-70b-versatile"
BATCH_SIZE = 50   # número de queries por batch

logger.info("✅ Ambiente carregado e cliente Groq configurado.")


2025-12-03 10:07:31 | INFO | ✅ Ambiente carregado e cliente Groq configurado.


In [2]:
# ============================================================
# NORMALIZAÇÃO DE QUERIES
# ============================================================

def normalize_query(q):
    """Remove espaços invisíveis + normaliza Unicode + limpa whitespace."""
    if pd.isna(q):
        return ""
    q = str(q)

    q = unicodedata.normalize("NFKC", q)    # normalização Unicode
    q = q.replace("\u200b", "")             # zero-width space
    q = q.replace("\xa0", " ")              # NBSP
    q = " ".join(q.split())                 # remover múltiplos espaços / trim

    return q


In [19]:
# ============================================================
# 1. LER FICHEIRO EXCEL
# ============================================================

def load_queries(path="TEST_queries_middle_east.xlsx"):
    df = pd.read_excel(path)
    # df = df.tail(5)  # PARA TESTES RÁPIDOS — descomentar se quiseres

    if "Query" not in df.columns:
        raise ValueError("A coluna 'Query' não existe no ficheiro.")

    # Remover linhas com queries vazias
    df = df.dropna(subset=["Query"]).copy()

    # print('Antes')
    # print(df["Query"])
    # print('---')
    # Normalização das queries
    df["Query"] = df["Query"].apply(normalize_query)
    # print('Depois')
    # print(df["Query"])

    # df["QueryID"] = range(1, len(df) + 1)
    return df

# TESTE: carregar e espreitar
df = load_queries()
logger.info(f"Total de linhas: {len(df)}")
df.head()


2025-12-03 10:19:56 | INFO | Total de linhas: 550


,Query,Value,Type,Country,Topic,Year
0,antihistaminique,100,Top,Algeria,Antihistamine,2024
1,antihistaminiques,27,Top,Algeria,Antihistamine,2024
2,مضادات الهيستامين,21,Top,Algeria,Antihistamine,2024
3,les antihistaminiques,12,Top,Algeria,Antihistamine,2024
4,anti histaminique,12,Top,Algeria,Antihistamine,2024


In [20]:
# ============================================================
# 2. REMOVER DUPLICADOS POR TEXTO
# ============================================================

def deduplicate_queries(df):
    df_unique = df[["Query"]].drop_duplicates().reset_index(drop=True)
    df_unique["UniqueID"] = range(1, len(df_unique) + 1)
    return df_unique

df_unique = deduplicate_queries(df)
logger.info(f"Queries únicas: {len(df_unique)}")
df_unique.head()


2025-12-03 10:20:04 | INFO | Queries únicas: 293


,Query,UniqueID
0,antihistaminique,1
1,antihistaminiques,2
2,مضادات الهيستامين,3
3,les antihistaminiques,4
4,anti histaminique,5


In [21]:
# ============================================================
# 3. BATCHING
# ============================================================

def chunk_list(df_or_list, n):
    """
    Se receber DataFrame, faz slicing por linhas.
    Se receber lista, faz slicing normal.
    """
    if isinstance(df_or_list, pd.DataFrame):
        for i in range(0, len(df_or_list), n):
            yield df_or_list.iloc[i:i+n]
    else:
        for i in range(0, len(df_or_list), n):
            yield df_or_list[i:i+n]

In [22]:
# ============================================================
# 4. PROMPT — SUPPLEMENT BOX 2A (COM JSON)
# ============================================================

def build_prompt_for_batch(batch):
    text = (
        "We have extracted several queries from GoogleTrends from the following "
        "nine countries: Algeria, Egypt, Iran, Iraq, Morocco, Pakistan, Saudi Arabia, "
        "Turkey and the United Arab Emirates.\n\n"
        "We want to identify which queries explicitly convey a question. "
        "These queries can be in English, Portuguese, French, Spanish or in any language "
        "spoken in the aforementioned countries.\n\n"
        "Below, you can find the list of queries.\n\n"
        "For each query, return a JSON array where each element has the form:\n"
        "{ \"query\": \"<query text>\", \"explicit_question\": \"YES\" or \"NO\" }\n\n"
        "Write \"YES\" only if the query explicitly conveys a question. "
        "Otherwise, write \"NO\".\n\n"
        "List of queries to be classified:\n"
    )

    for q in batch:
        text += f"- {q}\n"

    return text

# ============================================================
# 5. FUNÇÃO PARA EXTRAIR JSON DA RESPOSTA DO LLM
# ============================================================

def safe_json_extract(text):
    """
    Extrai JSON mesmo que exista texto antes/depois do array.
    Encontra o bloco entre '[' e ']' e tenta json.loads().
    """
    if not text:
        return None

    match = re.search(r"\[.*\]", text, re.DOTALL)
    if not match:
        return None

    candidate = match.group(0).strip()

    try:
        return json.loads(candidate)
    except json.JSONDecodeError:
        return None


In [23]:
test_prompt = build_prompt_for_batch(df_unique["Query"].head(3).tolist())
print(test_prompt)


We have extracted several queries from GoogleTrends from the following nine countries: Algeria, Egypt, Iran, Iraq, Morocco, Pakistan, Saudi Arabia, Turkey and the United Arab Emirates.

We want to identify which queries explicitly convey a question. These queries can be in English, Portuguese, French, Spanish or in any language spoken in the aforementioned countries.

Below, you can find the list of queries.

For each query, return a JSON array where each element has the form:
{ "query": "<query text>", "explicit_question": "YES" or "NO" }

Write "YES" only if the query explicitly conveys a question. Otherwise, write "NO".

List of queries to be classified:
- antihistaminique
- antihistaminiques
- مضادات الهيستامين



In [24]:
# ============================================================
# 6. FUNÇÃO PARA CLASSIFICAR UM BATCH COM O LLM
# ============================================================

def classify_batch_with_llm(batch):
    """
    Devolve lista de dicts: { "query": ..., "explicit_question": "YES"/"NO" }
    """
    prompt = build_prompt_for_batch(batch)

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    raw = response.choices[0].message.content

    # print(raw)  # para debug

    data = safe_json_extract(raw)

    if data is None:
        logger.warning(f"Falha ao extrair JSON. Resposta bruta: {raw}")
        return []

    cleaned = []
    for item in data:
        cleaned.append({
            "query": item.get("query", "").strip(),
            "explicit_question": item.get("explicit_question", "NO").strip().upper()
        })

    return cleaned

# TESTE COM UM MINI-BATCH
mini_batch = df_unique["Query"].head(5).tolist()
test_output = classify_batch_with_llm(mini_batch)
test_output


[{'query': 'antihistaminique', 'explicit_question': 'NO'},
 {'query': 'antihistaminiques', 'explicit_question': 'NO'},
 {'query': 'مضادات الهيستامين', 'explicit_question': 'NO'},
 {'query': 'les antihistaminiques', 'explicit_question': 'NO'},
 {'query': 'anti histaminique', 'explicit_question': 'NO'}]

In [25]:
# ============================================================
# 7. FUNÇÃO: EXECUTAR UMA RUN COMPLETA (RUN 1 / RUN 2)
# ============================================================

def run_llm_classification(df_unique, run_name):
    rows = df_unique[["UniqueID", "Query"]]
    results = []

    for batch_df in chunk_list(rows, BATCH_SIZE):
        logger.info(f"{run_name} - batch com {len(batch_df)} queries...")

        batch_queries = batch_df["Query"].tolist()
        batch_ids = batch_df["UniqueID"].tolist()

        out = classify_batch_with_llm(batch_queries)

        for uid, item in zip(batch_ids, out):
            results.append({
                "UniqueID": uid,
                run_name: item["explicit_question"]
            })

        time.sleep(1)

    return pd.DataFrame(results)

# ⚠️ Para testes usa um subset:
df_unique_test = df_unique.head(20).copy()
df_run1_test = run_llm_classification(df_unique_test, "LLM_run1")
df_run1_test.head()


2025-12-03 10:20:49 | INFO | LLM_run1 - batch com 20 queries...


,UniqueID,LLM_run1
0,1,NO
1,2,NO
2,3,NO
3,4,NO
4,5,NO


In [26]:
# ============================================================
# 8. CLASSIFICAÇÃO POR REGRAS MULTILINGUE
# ============================================================

QUESTION_WORDS = {
    "en": ["what","why","how","when","where","which","who","whom","whose",
           "is","are","am","do","does","did","can","could","should","would",
           "may","might","will","shall","have","has","had"],
    "es": ["qué","que","cómo","cuando","dónde","cual","cuál","quién","quien",
           "por qué","porque?","puedo","puede","pueden","debo","debe","deben"],
    "pt": ["o que","que","porquê","porque?","como","quando","onde","qual",
           "quais","quem","pode","podemos","devo","deves","devemos"],
    "fr": ["quoi","pourquoi","comment","quand","où","quel","quelle","quels",
           "que","qui","est-ce que","peux-tu","pouvez-vous"],
    "ar": ["ما","ماذا","كيف","لماذا","متى","أين","هل","كم"],
    "fa": ["چی","چه","چرا","چطور","کجا","کی","آیا"],
    "tr": ["ne","neden","nasıl","ne zaman","nerede","hangi","kim","mı","mi","mu","mü"],
    "ur": ["کیا","کیوں","کیسے","کب","کہاں","کون","آیا"]
}

ALL_QUESTION_WORDS = list({kw for kws in QUESTION_WORDS.values() for kw in kws})

PATTERNS = [
    r".*\? *$",
    r"^(\s*)(is|are|am|do|does|did|can|could|should|would|have|has|had)\b",
    r"^(\s*)est-ce que\b",
    r"\b(mı|mi|mu|mü)\?$",
    r"^(.*?)\b(há|há alguma|será que)\b",
    r"^(.*?)\b(acaso|será que)\b",
    r"^(\s*)آیا\b",
    r"^(\s*)هل\b",
]


def is_question_multilingual(text):
    text = (text or "").strip().lower()

    if text.endswith("?"):
        return True

    for kw in ALL_QUESTION_WORDS:
        if re.search(r"\b" + re.escape(kw) + r"\b", text):
            return True

    for pattern in PATTERNS:
        if re.search(pattern, text):
            return True

    return False


def apply_multilingual_rules(df_unique):
    df_unique["Rules"] = df_unique["Query"].apply(
        lambda x: "YES" if is_question_multilingual(x) else "NO"
    )
    return df_unique

# TESTE
df_unique_test = apply_multilingual_rules(df_unique.head(20).copy())
df_unique_test.head()


,Query,UniqueID,Rules
0,antihistaminique,1,NO
1,antihistaminiques,2,NO
2,مضادات الهيستامين,3,NO
3,les antihistaminiques,4,NO
4,anti histaminique,5,NO


In [ ]:
# ============================================================
# 9. MERGE FINAL
# ============================================================

def merge_results(df_original, df_unique, df_run1, df_run2):
    temp = df_unique.merge(df_run1, on="UniqueID", how="left")
    temp = temp.merge(df_run2, on="UniqueID", how="left")

    df_final = df_original.merge(
        temp[["Query", "Rules", "LLM_run1", "LLM_run2"]],
        on="Query",
        how="left"
    )

    return df_final

# Exemplo de pipeline completo em modo “manual”:

start_time = time.time()
logger.info("=== PIPELINE COMPLETO (TESTE EM SUBSET) ===")

df = load_queries()
df_unique = deduplicate_queries(df)

df_unique.to_excel("unique_test.xlsx", index=False)

# Para testes, subset:
df_unique_small = df_unique.tail(20).copy()

df_run1 = run_llm_classification(df_unique, "LLM_run1")
df_run1.to_excel("df_run1_test.xlsx", index=False)

df_unique_shuffled = df_unique.sample(frac=1, random_state=None).reset_index(drop=True)  # sample() função pandas usada para escolher linhas de forma aleatória
df_run2 = run_llm_classification(df_unique_shuffled, "LLM_run2")
# df_run2 = run_llm_classification(df_unique, "LLM_run2")
df_run2.to_excel("df_run2_test.xlsx", index=False)
df_unique = apply_multilingual_rules(df_unique)
df_unique.to_excel("df_unique_small_test.xlsx", index=False)

df_final_test = merge_results(df, df_unique, df_run1, df_run2)
df_final_test.to_excel("df_final_test.xlsx", index=False)
df_final_test.head()

elapsed = int(time.time() - start_time)
logger.info(f"Tempo total (subset): {elapsed} segundos")


2025-12-03 11:07:13 | INFO | LLM_run1 - batch com 50 queries...
2025-12-03 11:07:16 | INFO | LLM_run1 - batch com 50 queries...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kbfd6k7hf9fbsch4kf9n01ag` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99882, Requested 552. Please try again in 6m14.976s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}